In [22]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import END, MessageGraph

model = ChatOpenAI(temperature=0)

graph = MessageGraph()

In [23]:
document = {
    "title": "Chatbot",
    "description": "This is a chatbot that can chat with you.",
    "enough": 0.8,
    "ref_chat": ["a", "b", "c"],
}

In [24]:
def check_relevant_question(messages: list):
    print("messages", messages)
    if document.get("enough") > 0.7:
        return AIMessage("Yes, I can help you.")
    else:
        return AIMessage("I'm sorry, I can't help you.")

In [25]:
graph.add_node("oracle", model)
graph.add_edge("oracle", "rel_question")
graph.add_node("rel_question", check_relevant_question)
graph.add_edge("rel_question", END)

graph.set_entry_point("oracle")

runnable = graph.compile()

In [26]:
runnable.invoke(HumanMessage("Hello"))

messages [HumanMessage(content='Hello', id='be8180e2-5b56-429b-a3ab-e006e52849c9'), AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a33fa327-c72a-46c7-a69d-54eb8934f54e-0')]


[HumanMessage(content='Hello', id='be8180e2-5b56-429b-a3ab-e006e52849c9'),
 AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a33fa327-c72a-46c7-a69d-54eb8934f54e-0'),
 AIMessage(content='Yes, I can help you.', id='788137d9-ccf6-4af1-92b9-6af16b6d6aba')]

In [ ]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-tools-agent")

In [35]:
print(type(prompt))

<class 'langchain_core.prompts.chat.ChatPromptTemplate'>


In [42]:
prompt.invoke({"input": "hello", "agent_scratchpad": ["nothign much"]})

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant'), HumanMessage(content='hello'), HumanMessage(content='nothign much')])

In [ ]:
prompt.pretty_print()

In [57]:
graph2= MessageGraph()

In [58]:
from langchain_core.runnables import RunnablePassthrough

def find(messages: list):
    print("messages", messages)
    return AIMessage("I found it.")

passthrough = RunnablePassthrough()

graph2.add_node("receive", passthrough)
graph2.add_node("find", find)

graph2.add_edge("receive", "find")
graph2.add_edge("find", END)

graph2.set_entry_point("receive")

In [59]:
graph2.compile().invoke(["hi", "hello"])

messages [HumanMessage(content='hi', id='0e679f6c-c5f7-4e19-b17d-2dc59fcfcb19'), HumanMessage(content='hello', id='58abe5c3-2e05-4dc8-bfde-1fae06800397')]


[HumanMessage(content='hi', id='0e679f6c-c5f7-4e19-b17d-2dc59fcfcb19'),
 HumanMessage(content='hello', id='58abe5c3-2e05-4dc8-bfde-1fae06800397'),
 AIMessage(content='I found it.', id='bb87b5e5-4831-49e2-928c-c2200326d9c8')]

In [90]:
from langgraph.graph import END, MessageGraph, StateGraph
from typing import TypedDict, Annotated
from bson import ObjectId


def add_messages(left: list, right: list):
    """Add instead of overwriting"""
    return left + right


def add_ephemeral(old: dict, new: dict):
    """Add instead of overwriting"""
    return {**old, **new}


class DocumentState(TypedDict):
    _id: ObjectId
    phone_number: str
    name: str
    created_at: str
    updated_at: str
    user_info: object
    messages: Annotated[list, add_messages]
    questions: list
    ephemeral: Annotated[dict, add_ephemeral]

In [91]:
# model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, streaming=True)

# def print_name(state: DocumentState):
#     print(state["name"])
#     response = model.invoke([HumanMessage("Hello I am {}".format(state["name"]))])
#     return {"messages": [response]}

# graph.add_node("print_name", print_name)
# graph.add_edge("print_name", END)

# graph.set_entry_point("print_name")
# graph.compile().invoke({"_id": ObjectId(), "name": "John Doe"})

In [92]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

def pick_a_new_question(documentState: DocumentState):
    output_parser = StrOutputParser()

    unasked_questions = []
    for question in documentState["questions"]:
        if question["enough"] < documentState["ephemeral"]["enoughness_threshold"]:
            unasked_questions.append(question["content"])

    recent_messages = " / ".join(documentState["messages"][-4:])
    options = " / ".join(unasked_questions)

    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "you are a survey bot picking the next question based on the current conversation flow. You'll be provided with the last 4 messages and the list of unasked questions. Choose the next question.",
            ),
            ("human", "recent_messages: {recent_messages} \n options: {options}"),
        ]
    )

    chain = prompt | model | output_parser
    result = chain.invoke({"recent_messages": recent_messages, "options": options})

    return {"ephemeral": {"next_question": result}}

In [93]:
graph = StateGraph(DocumentState)
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, streaming=True)

graph.add_node("pick_a_new_question", pick_a_new_question)
graph.add_edge("pick_a_new_question", END)

graph.set_entry_point("pick_a_new_question")

graph.compile().invoke(
    {
        "_id": ObjectId(),
        "name": "John Doe",
        "messages": ["Hello", "Hi", "How are you?", "I am good."],
        "questions": [
            {"content": "What is your name?", "enough": 0.9},
            {"content": "How are you?", "enough": 0.8},
            {"content": "What is your favorite color?", "enough": 0.7},
        ],
        "ephemeral": {
            "key": 1,
            "key2": 2,
            "next_question": ["What is your name?"],
            "enoughness_threshold": 0.8,
        },
    }
)

/Users/minkijung/anaconda3/envs/survey_buddy/lib/python3.12/site-packages/pydantic/v1/main.py:998: RuntimeWarning: fields may not start with an underscore, ignoring "_id"
  warnings.warn(f'fields may not start with an underscore, ignoring "{f_name}"', RuntimeWarning)


{'_id': ObjectId('663c0b4e5ce313e2caf995c3'),
 'name': 'John Doe',
 'messages': ['Hello', 'Hi', 'How are you?', 'I am good.'],
 'questions': [{'content': 'What is your name?', 'enough': 0.9},
  {'content': 'How are you?', 'enough': 0.8},
  {'content': 'What is your favorite color?', 'enough': 0.7}],
 'ephemeral': {'key': 1,
  'key2': 2,
  'next_question': 'What is your favorite color?',
  'enoughness_threshold': 0.8}}

In [94]:
# conditional graph

msg = HumanMessage("Hello")

In [101]:
msg.to_json()

{'lc': 1,
 'type': 'constructor',
 'id': ['langchain', 'schema', 'messages', 'HumanMessage'],
 'kwargs': {'content': 'Hello', 'type': 'human'}}

In [ ]:
msg

In [120]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """hi {input}"""
)
prompt.format(input="John")

'hi John'

In [118]:
prompt.invoke({"input": "minki"})

StringPromptValue(text='hi <bound method Kernel.raw_input of <ipykernel.ipkernel.IPythonKernel object at 0x103d28c80>>')